In [6]:
!pip install transformers peft accelerate trl datasets bitsandbytes torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


ERROR: Could not find a version that satisfies the requirement trl (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for trl


In [3]:
!pip install torch


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0


In [5]:
!huggingface-cli whoami

amine-maazizi


In [1]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.5.1+cu121
CUDA version: 12.1


In [2]:
def check_gpu():
    if torch.cuda.is_available():
        print(f"GPU detected: {torch.cuda.get_device_name(0)}")
        print(f"CUDA version: {torch.version.cuda}")
        print(f"VRAM Available: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f} GB")
    else:
        print("No GPU detected - this will be VERY slow!")

check_gpu()

✅ GPU detected: NVIDIA GeForce RTX 4070 Laptop GPU
🔧 CUDA version: 12.1
💻 VRAM Available: 8.59 GB


In [3]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="processed_data.csv")

dataset = dataset.remove_columns(['Unnamed: 0', 'tokenized', 'input_ids', 'attention_mask', 'embeddings'])

dataset = dataset.rename_column("sentiment", "label")
dataset = dataset.rename_column("processed_text", "text")

dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

c:\Users\amine\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 31014 examples [00:03, 7806.10 examples/s]


LLama3 a pris très longements pour le fine-tunning ce qui a consommé toute la VRAM du GPU est à causé un crash. on optera pour phi-2, avec des optimisation.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

model_id = "microsoft/phi-2" 

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16  # Adjusted for lower VRAM usage
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# LoRA Configuration
peft_config = LoraConfig(
    r=8,  # Reduced from 16 to save VRAM
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(
    output_dir="./phi2-sentiment",
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Lowered batch size for 8GB+ VRAM GPUs
    gradient_accumulation_steps=2, 
    learning_rate=2e-4,  
    optim="adamw_torch",
    fp16=True,  # Enables mixed precision for lower memory usage
    logging_steps=20,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    report_to="none",
    gradient_checkpointing=True,  # Saves VRAM
)

c:\Users\amine\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amine\.cache\huggingface\hub\models--microsoft--phi-2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.23s/it]
c:\Users\amine\AppData\Lo

In [7]:
def format_instruction(sample):
    return [f"""### Text:
{sample['text']}

### Sentiment (positive/neutral/negative):
{sample['label']}"""]

# Trainer Setup
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    formatting_func=format_instruction,
    tokenizer=tokenizer,
    args=training_args
)

C:\Users\amine\AppData\Local\Temp\ipykernel_17404\2451259946.py:11: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Map: 100%|██████████| 6203/6203 [00:00<00:00, 28696.30 examples/s]


In [8]:
import time

start_time = time.time()
print("Starting training...")
trainer.train()
print(f"Training completed in {time.time()-start_time:.2f} seconds")

🚀 Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\amine\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,5.701600
2,No log,5.671593


c:\Users\amine\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\amine\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args

⏱️ Training completed in 177.42 seconds


In [10]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, "./phi2-sentiment/checkpoint-9")

def predict_sentiment(text):
    prompt = f"Analyze sentiment of this text:\n{text}\nSentiment:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=20)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
predict_sentiment("I'm very sad today")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Analyze sentiment of this text:\nI'm very sad today\nSentiment: Negative\n"

In [13]:
import re

def extract_sentiment(output_text):
    match = re.search(r"Sentiment:\s*(\w+)", output_text)
    return match.group(1) if match else "Unknown"


Negative


Pour recuperer les performances de notre modèle on suivera une méthode très simialire a celle utiliser pour le LLama brute.

In [20]:
model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = PeftModel.from_pretrained(base_model, "./phi2-sentiment/checkpoint-9")

Loading checkpoint shards: 100%|██████████| 2/2 [01:28<00:00, 44.07s/it]


In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
          

In [18]:
import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from peft import PeftModel

df = pd.read_csv("processed_data.csv")

X_text = df['processed_text']
y_encoded = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X_text, y_encoded, test_size=0.03, random_state=42)

In [26]:
def evaluate_phi_llm(X_test, y_test, label_mapping, log_interval=10):
    y_pred = []
    total_sentences = len(X_test)

    print(f"Starting evaluation with {total_sentences} sentences...")

    for i, sentence in enumerate(X_test):
        # Format input
        prompt = f"Analyze sentiment of this text:\n{sentence}\nSentiment:"
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        # Generate prediction
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=20)
        predicted_label = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract sentiment
        sentiment = extract_sentiment(predicted_label)
        print(sentiment)

        # Map to numerical label
        if "positive" in sentiment:
            y_pred.append(label_mapping["positive"])
        elif "negative" in sentiment:
            y_pred.append(label_mapping["negative"])
        else:
            y_pred.append(label_mapping["neutral"])

        if (i + 1) % log_interval == 0:
            print(f"Processed {i + 1}/{total_sentences} sentences...")

    print("Finished processing all sentences. Generating classification report...")

    report = classification_report(y_test, y_pred, output_dict=True)

    precision = report["weighted avg"]["precision"] * 100
    recall = report["weighted avg"]["recall"] * 100
    f1_score = report["weighted avg"]["f1-score"] * 100
    accuracy = accuracy_score(y_test, y_pred) * 100

    return {
        "Exactitude": f"{accuracy:.2f}%",
        "Précision": f"{precision:.2f}%",
        "Rappel": f"{recall:.2f}%",
        "Score F1": f"{f1_score:.2f}%"
    }

In [ ]:
label_mapping = {"positive": 1, "negative": 0, "neutral": 2}

metrics = evaluate_phi_llm(X_test, y_test, label_mapping)
for key, value in metrics.items():
    print(f"{key}: {value}")

Après 1 heure d'entraînement sur une *Nvidia GeForce RTX 4070 Ti*, les performances obtenues sont catastrophiques (seulement 10 % de précision !). Certes, l'entraînement est rapide, ce qui permet d'augmenter les exigences du fine-tuning, mais le test est trop long pour expérimenter efficacement avec des modifications.